# COMP0189: Applied Artificial Intelligence
## Week 3 (Model Selection and Assessment)

### After this week you will be able to ...
- encode categorical values with one-hot encoding
- know which encoding, scaling, and imputing method you should select in accordacne with the dataset characteristics
- impute missing data with KNN
- know how to streamline the preprocessing steps in advanced way (Pipeline and ColmnTransformer)
- select best model based on various cross-validation methods

### Acknowledgements
- https://scikit-learn.org/stable/
- https://archive.ics.uci.edu/ml/datasets/adult

In [ ]:
# Optional: For black-style code formatting within the notebook env.
!pip install nb-black

In [ ]:
# Optional: For black-style code formatting within the notebook env.
%load_ext nb_black

In [ ]:
# Previously, we used version 1.1.3 to use boston dataset as an example.
# Since there has been many updates in core functions after version 1.2,
# we are updating the scikit-learn to the latest version.
# Otherwise you will get some errors and FutureWarnings.
!pip install scikit-learn==1.2.1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Part 1: Encoding and Imputations

### Task 1: Load and Split the Dataset into train and test

In [ ]:
# TASK 1: Load Dataset
# We are going to use the same adult dataset as previous week.
# We have cleaned the dataset, but did not touch the missing values.
df = pd.read_csv("../data/clean_adult.csv")
df

### Task 2: Encode categorical variables (label/ordinal encoding & one-hot encoding)

### Important: We need special care when we are encoding categorical variables 

**1. Take care of the missing values**
- Beware not to encode missing values unless you are intending to do so.
- Sometimes you want to encode missing values to a separate cateogory. For example, when you want to predict if passengers of titanic had survived or not, missing data of certain features can actually have meaning, i.e., Cabin information can be missing because the body was not found.

**2. Know which encoding and scaling method you should select**
- If your categories are ordinal, then it makes sense to use a LabelEncoder with a MinMaxScaler. For example, you can encode [low, medium, high], as [1,2,3], i.e., distance between low to high is larger than that of medium and high.

- However, if you have non-ordinal categorical values, like [White, Hispanic, Black, Asian], then it would be better to use a OneHotEncoder instead of forcing ordinality with a LabelEncoder. Otherwise the algorithms you use (especially distance based algorithms like KNN) will make the assumption that the distance between White and Asian is larger than White and Hispanic, which is nonsensical.

**3. Split before you encode to avoid data leakage**
- Split the dataset before you encode your data. It is natural for algorithms to see unknown values in the validation/test set that was not appearing in the train set. `sklearn.preprocessing.OneHotEncoder` is good at handling these unknown categories (`handle_unknown` parameter).

- Discussion: What if you are certain about all the possible categories that can appear for each feature? Can you encode all the values before splitting the dataset into train and test set?


This notebook shows the three points in the following sections with examples.

### Task 2-1: Label Encoding (with missing values)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, 

label_encoder = LabelEncoder()

### Task 2-2: One Hot Encoding (with missing values imputation)

Tip 1: Impute the missing values (choose the right strategy) before doing OHE  
Tip 2: Try creating a separate dataframe with one-hot encoded columns and combine the dataframe with the original dataframe for the final one.

In [ ]:
from sklearn.impute import SimpleImputer

### Another (recommended) Method for Task 2
### Using `Pipeline` and `ColmnTransformer` to streamline the workflow.

We will do the same operation (with addition of scaling) but with much more succinct way.   
We use `Pipeline` and `ColmnTransformer`.  
`ColmnTransformer` is needed to apply different preprocessing steps to selected columns.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler

### Side Note: Data Imputation with KNN
For the adult dataset, missing data present only in categorical values, so imputing strategy that makes floating point may not make sense.
However, for continuous values, you can use various imputation strategies, such as taking simple mean or mean value from K nearest neighbors (KNN).
If you use `sklearn.imput.KNNImputer`, each sample’s missing values are imputed using the `mean` value from `n_neighbors` nearest neighbors found in the training set.
If you want to use `mode` value from neighbors (for categorical data imputation) you need to implement the imputer by yourself.

- `sklearn-pandas` package (https://pypi.org/project/sklearn-pandas/1.5.0/) provides `CategoricalImputer` class, which is suitable for such processing

Here, we use iris dataset to show how to use KNNImputer for continuous values

In [ ]:
from sklearn.datasets import load_iris
from sklearn.impute import KNNImputer

In [ ]:
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)

In [ ]:
# Applying a random mask to make missing data
mask = np.random.choice([True, False], size=iris_df.shape[0] * iris_df.shape[1])
mask[:500] = True
np.random.shuffle(mask)
mask = np.reshape(mask, iris_df.shape)
iris_df = iris_df.mask(~mask)

iris_df.isnull().sum()

In [ ]:
train_X, test_X = iris_df[:100], iris_df[100:]

In [ ]:
# It is important to impute the train and test set separately (not fitting KNN to test set) to avoid data leak.
imputer = KNNImputer(n_neighbors=5)
imputed_train_X = imputer.fit_transform(train_X)
imputed_test_X = imputer.transform(test_X)

In [ ]:
del iris, iris_df, mask, train_X, test_X, imputer, imputed_train_X, imputed_test_X

### Task 3: Create different preprocessing strategies of your own
Create different versions of X (X1 and X2) and y using different strategies for data imputation.  
Define different preprocessing strategies using the `Pipeline` and `ColmnTransformer` class

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
# Your explorations here

### Task 4: 
Train different models (KNN, SVM) to predict the y from the two versions of X (X1 and X2) with a fixed value of the regularization parameter. 
Centre and scale the data before training the models. Create tables or plots to show how accuracy varies for different imputation strategies or different models. 

### Task 4-1: Training KNN and SVM Models (with original preprocessing)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

### Task 4-2: Investigation
Create tables or plots to show how accuracy varies for different imputation strategies or different models. 
- What is the impact of the imputation strategy on the accuracy? 
- What is the impact of the model on the accuracy? 

## Part 2: Cross Validation (CV)

`scikit-learn` provides a nice visulaisation of various cross validation methods.  
This notebook focuses on the three main CV techniques: `KFold`, `StratifiedKFold`, `GroupKFold`, and `StratifiedGroupKFold` for optimizing models' hyper-parameters and to understand how different strategies might affect the models' performance. 

Visit: https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#visualizing-cross-validation-behavior-in-scikit-learn

![kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_006.png)
![stra-kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_003.png)
![group-kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_004.png)
![stra-group-kfold](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_010.png)

In [ ]:
from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    GroupKFold,
    StratifiedGroupKFold,
)

### Task 5
Now do CV to optimize the models hyperparameter using different k (2, 5, 10, 20) for the k-fold CV (KNN, SVM) to predict the y from the two versions of X (X1 and X2) and plot the model performance (mean accuracy and SD) for the different k. Centre and scale the data before training the models. 

Note: remember that the pre-processing steps, including data centering and scaling should be embedded in the CV. 

- How does the accuracy vary as the number of folds increase? 

### Task 6
Repeat task 5 using stratified CV with k=5. Compute the accuracy of the models implemented in task 5 (with k=5) and check if the model with stratified CV performs better across the different folds. Centre and scale the data before training the models. Create tables or plots to show these results.

### Task 7
Repeat task 5 using stratified group CV considering 'Race' as a group with k=5.
Compute the accuracy of the models implemented in task 5 (with k=5) and check if the model with stratified group CV performs better across the different races. 
Centre and scale the data before training the models. 
Create tables or plots to show these results. 